In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import math

df=pd.read_csv('house_data_train.csv', index_col = 0)
df = df.dropna()
df.columns
df.head(16200)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
9469,2422049107,20140508T000000,350000.0,4.0,1.75,2250,13515,1.0,0,0,...,8,2150,100,1940,0,98030,47.3789,-122.229,2150,12508
15048,6151800225,20150409T000000,475000.0,3.0,1.75,1850,26445,1.0,0,0,...,7,1850,0,1962,1977,98010,47.3412,-122.051,2110,23280
3728,2919701105,20141209T000000,422000.0,2.0,1.75,1320,2609,1.0,0,0,...,7,920,400,1938,0,98117,47.6878,-122.366,1200,4220
2552,4137060270,20150105T000000,313000.0,4.0,2.50,2460,10320,2.0,0,0,...,8,2460,0,1993,0,98092,47.2599,-122.215,2210,9024
3275,818500490,20141009T000000,153503.0,2.0,2.50,1240,3649,2.0,0,0,...,7,1240,0,1986,0,98003,47.3241,-122.322,1400,3721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2824079053,20150113T000000,440000.0,3.0,2.50,1910,66211,2.0,0,0,...,7,1910,0,1997,0,98024,47.5385,-121.911,2330,67268
12076,2193310320,20150306T000000,595000.0,4.0,2.50,2330,7064,1.0,0,0,...,8,1780,550,1984,0,98052,47.6955,-122.097,1740,8075
8709,7504010750,20140924T000000,649990.0,4.0,2.25,2130,11900,2.0,0,0,...,9,2130,0,1976,0,98074,47.6408,-122.058,2590,11900
18794,126059310,20141130T000000,1000000.0,3.0,2.25,3040,52302,1.0,0,0,...,9,3040,0,2005,0,98072,47.7635,-122.112,2070,38600


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'sqft_living', 'floors', 'waterfront', 'view', 'grade', 'sqft_above', 'sqft_basement']

X=df[features]
Y=df['price']

W=Y.std()

Y=np.array((Y-Y.mean())/Y.std())

M=X.mean()
X=X.apply(lambda rec:(rec-rec.mean())/rec.std(),axis=0)
M

bedrooms             3.367469
bathrooms            2.112701
sqft_living       2081.468580
sqft_lot         14987.381420
floors               1.492346
sqft_living       2081.468580
floors               1.492346
waterfront           0.007963
view                 0.237222
grade                7.656111
sqft_above        1789.841728
sqft_basement      291.626852
dtype: float64

###Split Data

In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle=False)

kfold = KFold(n_splits = 10, shuffle = True, random_state = 100)

lr = LinearRegression()

cv_scores = cross_val_score(lr, X_train, Y_train, cv = kfold)

###Linear Regression

In [4]:
def initialize(dim):
    b=random.random()
    a=np.random.rand(dim)
    return b,a

def predict_Y(b,a,X):
    return b + np.dot(X,a)

def get_cost(Y,Y_hat):
    Y_resd=Y-Y_hat
    return np.sum(np.dot(Y_resd.T,Y_resd))/len(Y-Y_resd)

def update_theta(x,y,y_hat,b_0,theta_o,learning_rate):
    db=(np.sum(y_hat-y)*2)/len(y)
    dw=(np.dot((y_hat-y),x)*2)/len(y)
    b_1=b_0-learning_rate*db
    theta_1=theta_o-learning_rate*dw
    return b_1,theta_1

def run_gradient_descent(X,Y,alpha,num_iterations):
    tolerance = 1e-8
    b,theta=initialize(X.shape[1])
    num_weight = len(features)
    weights = [f'weights_a{i}' for i in range(num_weight)]
    gd_iterations_df=pd.DataFrame(columns=['iteration','value_b', *weights, 'cost'])
    result_idx=0
    prev_cost = 0
    for iter_num in range(num_iterations):
        Y_hat=predict_Y(b,theta,X)
        this_cost=get_cost(Y,Y_hat)
        prev_b=b
        prev_theta=theta
        b,theta=update_theta(X,Y,Y_hat,prev_b,prev_theta,alpha)
        gd_iterations_df.loc[result_idx]=[iter_num,b, *theta[:num_weight] , this_cost]
        result_idx=result_idx+1   
        if (np.abs(this_cost - prev_cost)<tolerance):
          print(np.abs(this_cost - prev_cost))
          break
        prev_cost = this_cost
    return gd_iterations_df,b,theta
        
gd_iterations_df,b,theta=run_gradient_descent(X,Y,alpha=0.01,num_iterations=1000)
#gd_iterations_df.head(5)
gd_iterations_df

9.949880530601973e-09


,iteration,value_b,weights_a0,weights_a1,weights_a2,weights_a3,weights_a4,weights_a5,weights_a6,weights_a7,weights_a8,weights_a9,weights_a10,weights_a11,cost
0,0.0,2.660030e-02,0.262784,0.841381,0.856773,0.158240,0.726472,0.532180,0.247871,0.136924,0.146727,0.451285,0.384314,0.629228,12.525544
1,1.0,2.606830e-02,0.223598,0.785642,0.799546,0.146696,0.687715,0.474953,0.209115,0.131585,0.131149,0.402958,0.332604,0.607346,10.145805
2,2.0,2.554693e-02,0.188451,0.735503,0.748439,0.136310,0.652903,0.423846,0.174302,0.127008,0.117455,0.360054,0.286705,0.587275,8.243143
3,3.0,2.503599e-02,0.156917,0.690372,0.702800,0.126961,0.621617,0.378207,0.143016,0.123105,0.105439,0.321994,0.245996,0.568830,6.721654
4,4.0,2.453527e-02,0.128617,0.649723,0.662050,0.118536,0.593485,0.337457,0.114884,0.119800,0.094920,0.288260,0.209920,0.551848,5.504722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,958.0,1.045844e-10,-0.093331,-0.032560,0.351279,-0.047834,0.227640,0.026686,-0.250960,0.138773,0.124917,0.307362,0.104782,0.103652,0.404495
959,959.0,1.024927e-10,-0.093328,-0.032566,0.351278,-0.047834,0.227641,0.026685,-0.250959,0.138773,0.124916,0.307368,0.104780,0.103653,0.404495
960,960.0,1.004428e-10,-0.093325,-0.032572,0.351277,-0.047833,0.227642,0.026684,-0.250958,0.138774,0.124915,0.307375,0.104778,0.103655,0.404495
961,961.0,9.843398e-11,-0.093322,-0.032579,0.351277,-0.047833,0.227643,0.026684,-0.250958,0.138774,0.124914,0.307382,0.104777,0.103656,0.404495


###Pipeline

In [5]:
for i in range(1,4):

  steps = [("imp_mean", SimpleImputer()), 
           ("scaler", StandardScaler()),
           ("transform", PolynomialFeatures(degree = i)),
           ("liReg", LinearRegression())]

  pipeline = Pipeline(steps)
  pipeline.fit(X_train, Y_train)

  rscore_test = pipeline.score(X_test, Y_test)
  rscore_train = pipeline.score(X_train, Y_train)

  Y_pred = pipeline.predict(X_test)

  print(i, "test: ", rscore_test, "train: ", rscore_train)

1 test:  0.5957198024266375 train:  0.5942977317317192
2 test:  0.6617413060726104 train:  0.6774945138296509
3 test:  -0.43720038016019735 train:  0.7129296381243041


###cv_scores

In [6]:
print(cv_scores)

[0.61811525 0.59336534 0.56422316 0.54112621 0.56151695 0.60915203
 0.60434977 0.61398196 0.6016554  0.57755786]


###r2score_Linear Regression

In [7]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

normalize = MinMaxScaler()
X_scaled = normalize.fit_transform(X)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_scaled, Y)
print(lr.coef_, lr.intercept_, "\n")

Y_pred = lr.predict(X_scaled)
rscore = r2_score(Y, Y_pred)
mse = mean_squared_error(Y, Y_pred)
mae = mean_absolute_error(Y, Y_pred)

print('r2-score:', rscore, '\nmean squared error:', mse, '\nmean absolute error:', mae)

[-8.55075149e-01 -3.20912504e-01  1.22377905e+12 -1.27921078e+00
 -6.43095303e+13  7.87967422e+12  6.43095303e+13  1.55618387e+00
  6.48095791e-01  2.62153127e+00 -6.61715053e+12 -3.75673328e+12] -1.8917846679687502 

r2-score: 0.5953707541736983 
mean squared error: 0.4046042687123618 
mean absolute error: 0.4179250175322381


In [8]:
print("\nactual Price:",Y[2], "Predict Price:",Y_pred[2], "Differece:", Y[2]-Y_pred[2])


actual Price: -0.320283212536391 Predict Price: -0.4389038085937502 Differece: 0.1186205960573592


### Visualization for Linear Regression model

In [9]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import sklearn

# df = df.head(16200)

# # x = [df[:,1]].values.reshape(-1,1)

# sns.scatterplot(data = df, x = 'sqft_living', y = 'price')
# X_model = np.linspace(df['sqft_living'].min(), df['sqft_living'].max(), 50)
# Y_model = lr.predict(sklearn.preprocessing.StandardScaler().fit_transform(X_model.reshape(-1,1)))

# plt.plot(X_model,  Y_model, color = 'k')
# plt.legend(['training data', 'predicted model'])
# plt.show()

###Polynomial Regression

In [10]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

X = df['sqft_living'].values.reshape(-1, 1)
Y = df['price'].values.reshape(-1, 1)

#Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.reshape(-1,1))

# Poly transformation
order = 15
poly = PolynomialFeatures(degree=order, include_bias = False)
poly_features = poly.fit_transform(X_scaled.reshape(-1, 1))

# Modeling. 
lr = LinearRegression()
model = lr.fit(poly_features, Y)
print(model.coef_, model.intercept_)

#Prediction
Y_pred = model.predict(poly_features)
rscore = model.score(poly_features, Y)
mse = mean_squared_error(Y, Y_pred)
mae = mean_absolute_error(Y, Y_pred)

print('r2-score:', rscore.round(2), '\nmean squared error:', mse.round(2), '\nmean absolute error:', mae.round(2))
print("")
print("actual Price:",Y[0], "Predict Price:",Y_pred[0], "Differece:", Y[0]-Y_pred[0])

[[ 1.95312877e+05  9.66765981e+04 -2.42738598e+04 -6.06992588e+04
   3.65979561e+04  1.72600879e+04 -1.52015408e+04  1.01970475e+03
   1.77882682e+03 -6.94249225e+02  1.21436740e+02 -1.14791442e+01
   5.66921396e-01 -1.11900635e-02 -2.12089653e-05]] [489886.28079857]
r2-score: 0.56 
mean squared error: 61357048663.92 
mean absolute error: 165903.62

actual Price: [350000.] Predict Price: [528681.88445033] Differece: [-178681.88445033]


### Ridge Regression

In [11]:
pr = PolynomialFeatures(degree = i)

X_train_pr = pr.fit_transform(X_train)
X_test_pr = pr.fit_transform(X_test)

ridgeReg = Ridge(alpha=0.001, normalize=True)
ridgeReg.fit(X_train_pr, Y_train)

Ypred_train = ridgeReg.predict(X_train_pr)
Ypred_test = ridgeReg.predict(X_test_pr)

ridgeReg.score(X_train_pr, Y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


0.712232360565002

### Lasso Regression

In [12]:
pr = PolynomialFeatures(degree = i)
X_train_pr = pr.fit_transform(X_train)
X_test_pr = pr.fit_transform(X_test)

lassoReg = Lasso(alpha=100, max_iter=20000, normalize=True)
lassoReg.fit(X_train_pr, Y_train)

Ypred_train = lassoReg.predict(X_train_pr)
Ypred_test = lassoReg.predict(X_test_pr)

lassoReg.score(X_train_pr, Y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


0.0

### cv_score of Ridge and Lasso Regression

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

kfold = KFold(n_splits= 10, shuffle=True, random_state=100)

# lr = LinearRegression()

steps = [("imp_mean", SimpleImputer()),
          ("scaler" , StandardScaler()),
          ("tranform" , PolynomialFeatures(degree = i)),
          ("rReg" , Ridge(alpha=0, normalize=True)),
          # ("lRag", Lasso(alpha=100, max_iter=20000, normalize=True))
          ]
          
# r=Ridge(alpha=100, normalize=True)
# l=Lasso(alpha=100, max_iter=20000, normalize=True)

pipeline = Pipeline(steps)

cv_scores = cross_val_score(pipeline, X_train, Y_train, cv=kfold)

print(cv_scores)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipel

[0.65076001 0.73817346 0.53201946 0.55748443 0.6203095  0.64939624
 0.65056471 0.56987742 0.57242003 0.59159392]


### Visualization for Poly regression

In [14]:
#sns.scatterplot(data = df, x = 'sqft_living', y = 'price')
#x_model = np.linspace(df['sqft_living'].min(), df['sqft_living'].max(), 50)
#x_model_scaled = scaler.fit_transform(x_model.reshape(-1,1))

#x_model_poly  = poly.fit_transform(x_model_scaled)
#y_model = model.predict(x_model_poly)

#plt.plot(x_model,  y_model, color = 'k')
#plt.legend(['training data', 'predicted model with degree ' + str(order)])
#plt.show()

In [15]:
import joblib 
X = df['sqft_living'].values 
y = df['price'].values

scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X.reshape(-1,1))

poly = PolynomialFeatures(degree=6, include_bias=False) 
poly_features = poly.fit_transform(X_scaled.reshape(-1, 1))

lr = LinearRegression() 
model = lr.fit(poly_features, y)

joblib.dump(model,'best_model.pkl')

Y_pred = model.predict(poly_features) 
rscore = model.score(poly_features, y) 
mse = mean_squared_error(y, Y_pred) 
mae = mean_absolute_error(y, Y_pred)